In [ ]:
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(harmony))
suppressMessages(library(dplyr))
suppressMessages(library(destiny))
suppressMessages(library(cowplot))

In [ ]:
obj.rna <- readRDS("../../../snRNA/from_rico/ct_data/CM/CM_states.rds")

In [ ]:
obj.rna

In [ ]:
## add annotation for samples
df_sample_annotation <- read.csv("../../../snRNA/metadata/sample_annotation.csv", header = TRUE)
head(df_sample_annotation)

sample_to_region <- df_sample_annotation$region_novel
sample_to_patient <- df_sample_annotation$patient
sample_to_patient_region <- df_sample_annotation$patient_region_id
sample_to_patient_group <- df_sample_annotation$patient_group

names(sample_to_region) <- df_sample_annotation$sample_id
names(sample_to_patient) <- df_sample_annotation$sample_id
names(sample_to_patient_region) <- df_sample_annotation$sample_id
names(sample_to_patient_group) <- df_sample_annotation$sample_id

In [ ]:
obj.rna@meta.data$region <- stringr::str_replace_all(obj.rna@meta.data$orig.ident, sample_to_region)
obj.rna@meta.data$patient <- stringr::str_replace_all(obj.rna@meta.data$orig.ident, sample_to_patient)
obj.rna@meta.data$patient_region_id <- stringr::str_replace_all(obj.rna@meta.data$orig.ident, sample_to_patient_region)
obj.rna@meta.data$patient_group <- stringr::str_replace_all(obj.rna@meta.data$orig.ident, sample_to_patient_group)

In [ ]:
df_cell_count <- as.data.frame(obj.rna@meta.data) %>%
    group_by(orig.ident) %>%
    summarise(count = n()) %>%
    arrange(desc(count))

df_cell_count

In [ ]:
# we remove samples with less than 400 cells
df_cell_count <- subset(df_cell_count, count > 400)
obj.rna.sub <- subset(obj.rna, orig.ident %in% df_cell_count$orig.ident)

In [ ]:
obj.rna.sub

In [ ]:
obj.rna.sub <- obj.rna.sub %>% 
        NormalizeData() %>%
        FindVariableFeatures() %>%
        ScaleData() %>%
        RunPCA(verbose = FALSE) %>%
        RunUMAP(dims = 1:30)

In [ ]:
options(repr.plot.height = 5, repr.plot.width = 20)

p1 <- DimPlot(obj.rna.sub, reduction = "umap", group.by = "orig.ident")
p2 <- DimPlot(obj.rna.sub, reduction = "umap", group.by = "patient")
p3 <- DimPlot(obj.rna.sub, reduction = "umap", group.by = "region")
p4 <- DimPlot(obj.rna.sub, reduction = "umap", group.by = "patient_group", label = TRUE)

patchwork::wrap_plots(list(p1, p2, p3, p4), nrow = 1)

In [ ]:
obj.rna.sub <- RunHarmony(obj.rna.sub, 
                      group.by.vars = c("orig.ident", "patient", "region", "batch"),
                     reduction = "pca", 
                      max.iter.harmony = 30, 
                      dims.use = 1:30,
                     project.dim = FALSE,
                     plot_convergence = TRUE)


obj.rna.sub <- RunUMAP(obj.rna.sub, 
               dims = 1:30, 
               reduction = 'harmony',
               reduction.name = "umap_harmony",
               reduction.ke = 'umapharmony_',
              verbose = FALSE,
                   min.dist = 0.4)

In [ ]:
options(repr.plot.height = 5, repr.plot.width = 20)

p1 <- DimPlot(obj.rna.sub, reduction = "umap_harmony", group.by = "orig.ident")
p2 <- DimPlot(obj.rna.sub, reduction = "umap_harmony", group.by = "patient")
p3 <- DimPlot(obj.rna.sub, reduction = "umap_harmony", group.by = "region")
p4 <- DimPlot(obj.rna.sub, reduction = "umap_harmony", group.by = "patient_group", label = TRUE)

patchwork::wrap_plots(list(p1, p2, p3, p4), nrow = 1)

In [ ]:
# opt_state_cols <- ArchR::paletteDiscrete(obj.rna$opt_state)

In [ ]:
# cols <- c("control_P1" = "#440154FF", "control_P18" = "#440154FF", 
#           "control_P7" = "#440154FF", "control_P8" = "#440154FF",
#          "FZ_P15" = "#35B779FF", "FZ_P16" = "#35B779FF",
#          "FZ_P17" = "#35B779FF", "FZ/GT_P16" = "#35B779FF",
#          "FZ/GT_P4" = "#35B779FF", "GT/IZ_P10" = "#FDE725FF",
#          "GT/IZ_P14" = "#FDE725FF", "IZ_P11" = "#FDE725FF",
#          "IZ_P16" = "#FDE725FF", "IZ_P17" = "#FDE725FF",
#          "IZ_P3" = "#FDE725FF", "IZ/BZ_P2" = "#FDE725FF",
#          "IZ/GT_P10" = "#FDE725FF", "IZ/GT_P16" = "#FDE725FF",
#          "RZ_P10" = "#31688EFF", "RZ_P12" = "#31688EFF",
#          "RZ_P3" = "#31688EFF", "RZ_P6" = "#31688EFF",
#          "RZ/BZ_P13" = "#31688EFF", "RZ/BZ_P2" = "#31688EFF",
#          "RZ/BZ_P3" = "#31688EFF", "RZ/FZ_P5" = "#31688EFF",
#          "RZ/GT_P9" = "#31688EFF")

In [ ]:
# df_ct <- obj.rna.sub@meta.data %>%
#     as.data.frame() %>%
#     group_by(patient, opt_state) %>%
#     summarise(counts = n()) %>%
#     mutate(cell_proportion = counts / sum(counts))

# p <- ggplot(df_ct, aes(patient, cell_proportion, fill=opt_state)) + 
#     geom_bar(stat="identity", position = position_stack(reverse = TRUE)) +
#     scale_fill_manual(values = opt_state_cols) +
#     theme_cowplot() +
#     xlab("") + ylab("") +
#     theme(axis.text.x = element_text(angle = 60, hjust = 1))
    

# options(repr.plot.width = 10, repr.plot.height = 6)

# p

In [ ]:
suppressMessages(library(Nebulosa))

options(repr.plot.height = 10, repr.plot.width = 10)

p1 <- plot_density(obj.rna.sub, features="NPPA", reduction="umap_harmony")
p2 <- plot_density(obj.rna.sub, features="NPPB", reduction="umap_harmony")
p3 <- plot_density(obj.rna.sub, features="ANKRD1", reduction="umap_harmony")
p4 <- plot_density(obj.rna.sub, features="MYO18B", reduction="umap_harmony")

patchwork::wrap_plots(list(p1, p2, p3, p4), nrow = 2)

In [ ]:
## find marker
Idents(obj.rna.sub) <- "opt_state"

all.markers <- FindAllMarkers(obj.rna.sub, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

df <- all.markers %>%
    group_by(cluster) %>%
    slice_max(n = 10, order_by = avg_log2FC)

# Dot plots - the size of the dot corresponds to the percentage of cells expressing the
# feature in each cluster. The color represents the average expression level
options(repr.plot.height = 4, repr.plot.width = 15)
DotPlot(obj.rna.sub, features = df$gene) + RotatedAxis()

if(!dir.exists("../data/snRNA")){
    dir.create("../data/snRNA")
}

markerList <- split(all.markers, all.markers$cluster)

for(i in 1:length(markerList)){
    markerList[[i]] <- markerList[[i]][order(-markerList[[i]]$avg_log2FC), ]
}

WriteXLS::WriteXLS(markerList,
                   ExcelFileName = "../data/snRNA/MarkerGenes.xlsx",
                   SheetNames = names(markerList))

saveRDS(all.markers, "../data/snRNA/MarkerGenes.rds")

In [ ]:
# df <- as.data.frame(obj.rna.sub@reductions$harmony@cell.embeddings)
# data <- as.ExpressionSet(df)
# dm <- DiffusionMap(data,
#                    verbose = TRUE)
# dpt <- DPT(dm)
# plot(dpt)

# embedding <- as.data.frame(dm)[, c("DC1", "DC2")]
# rownames(embedding) <- colnames(obj.rna.sub)
# obj.rna.sub[['dm']] = CreateDimReducObject(embeddings = as.matrix(embedding),
#                                               key = 'DC_',
#                                               assay = 'RNA')
# options(repr.plot.height = 8, repr.plot.width = 8)
# DimPlot(obj.rna.sub, reduction = 'dm', pt.size = 0.5, group.by = "opt_state")

In [ ]:
saveRDS(obj.rna.sub, "../data/snRNA/snRNA.rds")

In [ ]:
sessionInfo()